***Matrix Factorization***

Steps :

1/ Pick random numbers for each customer and each product 

2/ Find a score for each customer and product

3/ Rank according to these scores 

4/ Tweak the customer and user vectors to get better rankings

Libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
train = pd.read_parquet("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_train_with_alphanumeric_dummy_ids.parquet")

In [ ]:
print(train.shape)
print(train.head())

(165042, 2)
               dummyUserId  productId
0  b'PIXcm7Ru5KmntCy0yA1K'   10524048
1  b'd0RILFB1hUzNSINMY4Ow'    9137713
2  b'Ebax7lyhnKRm4xeRlWW2'    5808602
3  b'vtigDw2h2vxKt0sJpEeU'   10548272
4  b'r4GfiEaUGxziyjX0PyU6'   10988173


In [ ]:
valid = pd.read_parquet("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_valid_with_alphanumeric_dummy_ids.parquet")

In [ ]:
dummy_users = pd.read_csv("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_dummy_users_with_alphanumeric_dummy_ids.csv", header=None).values.flatten().astype(str)

In [ ]:
products = pd.read_csv("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_productIds.csv", header=None).values.flatten().astype(int)

In [ ]:
dummy_users

array(['pmfkU4BNZhmtLgJQwJ7x', 'UDRRwOlzlWVbu7H8YCCi',
       'QHGAef0TI6dhn0wTogvW', ..., 'lcORJ5hemOZc1iGo9z7k',
       '5CqDquDAszqJp27P7AL8', 'SSPNYxJMfuKhoe1dg24m'], dtype='<U20')

Define The Recommender Model :

In [ ]:
print(dummy_users)
print(len(dummy_users))

['pmfkU4BNZhmtLgJQwJ7x' 'UDRRwOlzlWVbu7H8YCCi' 'QHGAef0TI6dhn0wTogvW' ...
 'lcORJ5hemOZc1iGo9z7k' '5CqDquDAszqJp27P7AL8' 'SSPNYxJMfuKhoe1dg24m']
43607


In [ ]:
len_users = len(set(dummy_users))

In [ ]:
len_users

43607

In [ ]:
products

array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

In [ ]:
user_embedding = tf.keras.layers.Embedding(len(dummy_users), 6)

In [ ]:
product_embedding = tf.keras.layers.Embedding(len(products), 6)

In [ ]:
#try example user 
user_ex = user_embedding(2)
product_ex = product_embedding(10)
#compute score ( dot product)
tf.squeeze(tf.matmul(tf.expand_dims(product_ex, axis=0), tf.transpose(tf.expand_dims(user_ex, axis=0))))

<tf.Tensor: shape=(), dtype=float32, numpy=0.0020714889>

In [ ]:
tf.tensordot(user_ex, product_ex, axes=[[0], [0]])

<tf.Tensor: shape=(), dtype=float32, numpy=0.0020714889>

https://www.edgeimpulse.com/blog/scale-your-tinyml-application-with-edge-impulse-and-azure-iot-edge

In [ ]:
example_prod = tf.constant([1, 88, 9])

In [ ]:
 product_embedding(example_prod)

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[-0.03893924,  0.01756604, -0.00629457,  0.02782004, -0.03665216,
        -0.02876104],
       [ 0.00955944, -0.03944452, -0.03754163, -0.03430908,  0.01168757,
         0.045934  ],
       [-0.03682883,  0.04547347, -0.00124203,  0.02301201,  0.03769879,
        -0.02672557]], dtype=float32)>

In [ ]:
user_embedding(1)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 0.00180739, -0.00803481, -0.04191581, -0.02918632,  0.04782642,
        0.04224632], dtype=float32)>

In [ ]:
product_embedding(example_prod)

<tf.Tensor: shape=(3, 6), dtype=float32, numpy=
array([[-0.03893924,  0.01756604, -0.00629457,  0.02782004, -0.03665216,
        -0.02876104],
       [ 0.00955944, -0.03944452, -0.03754163, -0.03430908,  0.01168757,
         0.045934  ],
       [-0.03682883,  0.04547347, -0.00124203,  0.02301201,  0.03769879,
        -0.02672557]], dtype=float32)>

In [ ]:
tf.tensordot(user_embedding(1), product_embedding(example_prod), axes=[[0], [1]])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.00372763,  0.00540867, -0.00037757], dtype=float32)>

Problem : 
- products are not continuous numbers 

- dummy users are strings  

Solution : 
set a lookup dictionary 

In [ ]:
products

array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

In [ ]:
dummy_users

array(['pmfkU4BNZhmtLgJQwJ7x', 'UDRRwOlzlWVbu7H8YCCi',
       'QHGAef0TI6dhn0wTogvW', ..., 'lcORJ5hemOZc1iGo9z7k',
       '5CqDquDAszqJp27P7AL8', 'SSPNYxJMfuKhoe1dg24m'], dtype='<U20')

Tensorflow Hashtable :


In [ ]:
tf.range(15)

<tf.Tensor: shape=(15,), dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=int32)>

In [ ]:
#mapping each product id with an integer in the range (0, len(product))
product_table =  tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(tf.constant(products, dtype=tf.int32), tf.range(len(products))),
    default_value=-1)

In [ ]:
product_table

In [ ]:
product_table.lookup(tf.constant([[8650774, 11927550], [12058614,9961521 ]]))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[    0, 29695],
       [29693,     2]], dtype=int32)>

In [ ]:
product_table.lookup(tf.constant([8650774]))

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>

In [ ]:
user_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(tf.constant(dummy_users), tf.range(len(dummy_users))),
    default_value=-1)

In [ ]:
print(tf.range(5))
print(range(5))


tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
range(0, 5)


In [ ]:
class SimpleRecommender(tf.keras.Model):
  def __init__(self, dummy_users, products, length_of_embedding):
    super(SimpleRecommender, self).__init__()

    self.products = tf.constant(products, dtype=tf.int32)
    self.dummy_users = tf.constant(dummy_users,dtype=tf.string)

    self.dummy_users_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.dummy_users, range(len(self.dummy_users))),
    default_value=-1)
    self.products_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.products, range(len(self.products))),
    default_value=-1)

    self.user_embedding =  tf.keras.layers.Embedding(len(self.dummy_users), length_of_embedding)
    self.product_embedding = tf.keras.layers.Embedding(len(self.products), length_of_embedding)

    self.dot = tf.keras.layers.Dot(axes=-1)

  def call(self, inputs):
    user = inputs[0]
    products = inputs[1]

    user_embedding_index = self.dummy_users_table.lookup(user)
    product_embedding_index = self.products_table.lookup(products)

    user_embedding_values = self.user_embedding(user_embedding_index)
    product_embedding_values = self.product_embedding(product_embedding_index)

    return tf.squeeze(self.dot([user_embedding_values, product_embedding_values]), axis=1)

  @tf.function
  def call_item_item(self, product):
    """
    find item to item similarity 
    """
    product_x = self.products_table.lookup(product)
    product_embed =  tf.expand_dims(self.product_embedding(product_x), axis=0)
    print(product_embed)
    all_embed = tf.expand_dims(self.product_embedding.embeddings, axis=0)
    scores = self.dot([product_embed, all_embed])
    scores = tf.reshape(scores, [-1])
    top_scores, top_indices = tf.math.top_k(scores, k=100)
    top_ids = tf.gather(self.products, top_indices)
    return top_ids, top_scores

    

In [ ]:
train

,dummyUserId,productId
0,b'PIXcm7Ru5KmntCy0yA1K',10524048
1,b'd0RILFB1hUzNSINMY4Ow',9137713
2,b'Ebax7lyhnKRm4xeRlWW2',5808602
3,b'vtigDw2h2vxKt0sJpEeU',10548272
4,b'r4GfiEaUGxziyjX0PyU6',10988173
...,...,...
165037,b'7Eom5Ancozj01ozGxAMK',9071435
165038,b'zi9vZETHqSIZK0TM2nZc',10413104
165039,b'fVCveec9P946asY5wqGm',9859881
165040,b'VJtfpw602SZHh2qwarK4',10809487


In [ ]:
sr1 = SimpleRecommender(dummy_users, products, 15)

In [ ]:
dummy_users

array(['pmfkU4BNZhmtLgJQwJ7x', 'UDRRwOlzlWVbu7H8YCCi',
       'QHGAef0TI6dhn0wTogvW', ..., 'lcORJ5hemOZc1iGo9z7k',
       '5CqDquDAszqJp27P7AL8', 'SSPNYxJMfuKhoe1dg24m'], dtype='<U20')

In [ ]:
products


array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

In [ ]:
sr1([tf.constant([['pmfkU4BNZhmtLgJQwJ7x'], ['UDRRwOlzlWVbu7H8YCCi']]), tf.constant([[8650774, 11927550], [12058614,9961521 ]])])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 0.0017606 , -0.00039608],
       [ 0.00457848, -0.00505342]], dtype=float32)>

In [ ]:
train[['dummyUserId']].values.shape

(165042, 1)

In [ ]:
train['dummyUserId'].values.shape

(165042,)

In [ ]:
dummy_user_tensor = tf.constant(train[['dummyUserId']].values, dtype=tf.string)
product_tensor = tf.constant(train[['productId']].values, dtype=tf.int32)


In [ ]:
dummy_user_tensor

<tf.Tensor: shape=(165042, 1), dtype=string, numpy=
array([[b'PIXcm7Ru5KmntCy0yA1K'],
       [b'd0RILFB1hUzNSINMY4Ow'],
       [b'Ebax7lyhnKRm4xeRlWW2'],
       ...,
       [b'fVCveec9P946asY5wqGm'],
       [b'VJtfpw602SZHh2qwarK4'],
       [b'D8BlXaKnJHXmxgwfSZzE']], dtype=object)>

**Create Dataset**

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor))
for x, b in dataset:
  print(x)
  print(b)
  break

tf.Tensor([b'PIXcm7Ru5KmntCy0yA1K'], shape=(1,), dtype=string)
tf.Tensor([10524048], shape=(1,), dtype=int32)


In [ ]:
#search for the most wanted product :
train['productId'].value_counts()

10308557    502
10308818    379
10402698    355
9276055     341
10309914    285
           ... 
8517596       1
8152130       1
9531312       1
11378606      1
8984611       1
Name: productId, Length: 29696, dtype: int64

In [ ]:
#list of products that user didn't purshase 

random_negatives_index = tf.random.uniform((7,), minval=0, maxval=len(products), dtype=tf.int32)

In [ ]:
tf.gather(products, random_negatives_index)

<tf.Tensor: shape=(7,), dtype=int64, numpy=
array([10602338, 11702935, 11027444,  8548776,  9856218,  9554187,
       11760109])>

In [ ]:
class Mapper():
  def __init__(self, possible_products, num_negative_products):
    self.num_possible_products = len(possible_products)
    self.possible_poducts_tensor = tf.constant(possible_products, dtype=tf.int32)
    self.num_negative_products = num_negative_products
    #create a y tensor to hanle 
    self.y = tf.one_hot(0, self.num_negative_products +1)
  def __call__(self, user, product):
    random_negatives_index = tf.random.uniform((self.num_negative_products,), minval=0, maxval=self.num_possible_products, dtype=tf.int32)
    negatives = tf.gather(self.possible_poducts_tensor, random_negatives_index)
    candidates = tf.concat([product, negatives], axis=0)
    return (user, candidates),  self.y


In [ ]:
products

array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor)).map(Mapper(products, 6))

In [ ]:
for x, y in dataset:
  print(x)
  print(y)
  break

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'PIXcm7Ru5KmntCy0yA1K'], dtype=object)>, <tf.Tensor: shape=(7,), dtype=int32, numpy=
array([10524048, 12261533, 11475281, 11620213,  9780546, 11905841,
       11988847], dtype=int32)>)
tf.Tensor([1. 0. 0. 0. 0. 0. 0.], shape=(7,), dtype=float32)


In [ ]:
def get_dataset(df, products, num_negatives):
  dummy_user_tensor = tf.constant(df[['dummyUserId']].values, dtype=tf.string)
  product_tensor = tf.constant(df[['productId']].values, dtype=tf.int32)
  dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor)).map(Mapper(products, num_negatives))
  dataset = dataset.batch(1024)
  return dataset

In [ ]:
for x, y in get_dataset(train, products, 4):
  print(x)
  print(y)
  break

(<tf.Tensor: shape=(1024, 1), dtype=string, numpy=
array([[b'PIXcm7Ru5KmntCy0yA1K'],
       [b'd0RILFB1hUzNSINMY4Ow'],
       [b'Ebax7lyhnKRm4xeRlWW2'],
       ...,
       [b'xuX9n8PHfSR0AP3UZ8ar'],
       [b'iNnxsPFfOa9884fMjVPJ'],
       [b'aD8Mn12im8lFPzXAY41P']], dtype=object)>, <tf.Tensor: shape=(1024, 5), dtype=int32, numpy=
array([[10524048, 10067492, 12537830, 11626951,  8439664],
       [ 9137713, 11395149,  9225924, 10202700, 11176029],
       [ 5808602, 11423661, 10275196,  9978502, 10130877],
       ...,
       [11541336, 12624773, 11776802, 12756021, 11515855],
       [ 7779232,  8564990, 10493549, 10489415, 12475384],
       [ 4941259, 10211312, 12314045, 10239067, 10563207]], dtype=int32)>)
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(1024, 5), dtype=float32)


**Train A Model**

- compile a model
- set loss and evaluation metric

In [ ]:
for x, y in get_dataset(train, products, 100):
  print(x)
  print(y)
  break

(<tf.Tensor: shape=(1024, 1), dtype=string, numpy=
array([[b'PIXcm7Ru5KmntCy0yA1K'],
       [b'd0RILFB1hUzNSINMY4Ow'],
       [b'Ebax7lyhnKRm4xeRlWW2'],
       ...,
       [b'xuX9n8PHfSR0AP3UZ8ar'],
       [b'iNnxsPFfOa9884fMjVPJ'],
       [b'aD8Mn12im8lFPzXAY41P']], dtype=object)>, <tf.Tensor: shape=(1024, 101), dtype=int32, numpy=
array([[10524048,  9267613, 12109701, ...,  8845079,  9982803, 11198969],
       [ 9137713, 11592185, 10256214, ..., 11344398, 12113702, 11597778],
       [ 5808602, 11060961, 10125748, ..., 10824946,  9067777, 13298085],
       ...,
       [11541336, 12401147, 10239276, ...,  8193592, 12543667, 10387469],
       [ 7779232,  9790734,  8651980, ..., 13117754,  9806484, 10927861],
       [ 4941259, 12654470, 10768032, ..., 12159378, 11455350, 10636200]],
      dtype=int32)>)
tf.Tensor(
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]], shape=(1024, 101), 

In [ ]:
model = SimpleRecommender(dummy_users, products, 15)
#categorical crossentropy converts scores to probabilities
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.SGD(100), metrics=tf.keras.metrics.CategoricalAccuracy())
#to check wether our dataset is useful in the real world we pass in a validation dataset
model.fit(get_dataset(train, products, 100), validation_data=get_dataset(valid, products, 100), epochs= 5)


Epoch 1/5
162/162 [==============================] - 20s 118ms/step - loss: 4.6151 - categorical_accuracy: 0.0138 - val_loss: 4.6151 - val_categorical_accuracy: 0.0182
Epoch 2/5
162/162 [==============================] - 19s 117ms/step - loss: 4.6119 - categorical_accuracy: 0.0841 - val_loss: 4.6147 - val_categorical_accuracy: 0.0460
Epoch 3/5
162/162 [==============================] - 19s 118ms/step - loss: 4.5975 - categorical_accuracy: 0.1914 - val_loss: 4.6064 - val_categorical_accuracy: 0.0714
Epoch 4/5
162/162 [==============================] - 19s 117ms/step - loss: 4.5263 - categorical_accuracy: 0.2437 - val_loss: 4.5614 - val_categorical_accuracy: 0.0941
Epoch 5/5
162/162 [==============================] - 20s 122ms/step - loss: 4.3133 - categorical_accuracy: 0.2466 - val_loss: 4.4265 - val_categorical_accuracy: 0.1079


In [ ]:
test_product = products[0]
test_product

8650774

In [ ]:
#test product :
print("Recs for item {}: {}".format(test_product, model.call_item_item(tf.constant(test_product, dtype=tf.int32))))

Tensor("ExpandDims:0", shape=(1, 15), dtype=float32)
Recs for item 8650774: (<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([ 8941836, 11541077, 11953072,  9109184, 10062595, 12050426,
       11409811, 10357545, 10102320, 11374873, 10867650, 10390716,
        9109088, 11311936, 11691428, 10487343,  9565144,  9071435,
       10555920, 10273738,  8661992, 12746059, 12115410, 10131359,
       11080858, 10371371, 10464977, 11100314, 10375474, 11832342,
        8840743,  9407068, 10581397,  6775544,  9189046, 10234617,
       10437093,  9921383, 10710521,  8768733, 10637116, 11462829,
       11448828, 10104809, 10909750, 12201985,  8927246, 11198985,
       11392806, 12155249, 11493983, 11638705, 10715306, 10357533,
       12111280, 11903387, 10636823, 11054197, 11700318, 11777393,
       10339012,  8469882,  7761008, 12235188, 12259464, 13000935,
       11409896, 12657366, 11211459, 10877926, 11273515, 10260333,
       11427852, 10424246,  9539647, 12420888, 11396817, 11076588,
      

Load Model

In [ ]:
#the function call_item_item is saved as a graph in the protobuf file (saved_model.pb)

In [ ]:
model_path = "models/recommender/1"

In [ ]:
input_signature = tf.TensorSpec(shape=(), dtype=tf.int32)
signatures = {"call_item_item": model.call_item_item.get_concrete_function(input_signature)}

Tensor("ExpandDims:0", shape=(1, 15), dtype=float32)


In [ ]:
tf.saved_model.save(model, model_path, signatures=signatures)

Tensor("ExpandDims:0", shape=(1, 15), dtype=float32)
INFO:tensorflow:Assets written to: models/recommender/1/assets


In [ ]:
imported_model = tf.saved_model.load(model_path)
list(imported_model.signatures.keys())

['call_item_item']

In [ ]:
products

array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

In [ ]:
result_tensor = imported_model.signatures['call_item_item'](tf.constant([8650]))
result_tensor

{'output_0': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([ 8650774,  9306139,  9961521, 13238328, 10485819, 13238336,
        13238341, 13238349,  9175127, 11141217, 11141222, 11141223,
        11141224, 11141226,  9568368,  8388728,  9568381,  9568388,
         9830537,  9437342,  9961638, 12714153, 13107382, 11403450,
         9044155, 13107389, 10748117, 10748127, 10748128, 10748131,
        10748137, 10748138,  8782062,  8782064, 12058867, 10879220,
        10223863, 10486008, 10879225, 10879227, 10879230, 10879231,
        12321027,  8388869, 10879238, 10879241, 10354955, 10354958,
        10354961, 10354965, 10354966, 10354971, 10354972, 10354973,
         7340321, 10354977, 10354978, 11927847, 13369677,  8520014,
        10223954, 13369684, 10223957, 10223960, 11403612, 11403613,
        11272541, 11796832, 11796833, 11534690, 11272546, 11272553,
        11403626,  8651167, 13107617, 11010469,  8651175, 11010474,
         8651182, 11010484, 12976579, 11010502, 11010507,

In [ ]:
!curl --header "Content-Type: application/json" --request POST --data '{"signature_name":"call_item_item","inputs": {"product": [8650774] } }' http://localhost:8501/v1/models/recommender:predict


curl: (7) Failed to connect to localhost port 8501: Connection refused


In [ ]:

!curl http://localhost:8501/v1/models/recommender/metadata



curl: (7) Failed to connect to localhost port 8501: Connection refused


In [ ]:
valid.head()

,dummyUserId,productId
0,b'I4Yc5Ztur3UNwY5SdvDh',10093853
1,b'nhWgcxEVY7jQ3MvvNxWL',12306408
2,b'3vriQXKwG095rvR1MSrz',11858310
3,b'MA8KmOxkGd1JQ42GXDGO',10072124
4,b'vax7VgJnswdiC8iHZSCi',10596405


In [ ]:
!unzip models.zip

unzip:  cannot find or open models.zip, models.zip.zip or models.zip.ZIP.


In [ ]:
imported_model = tf.saved_model.load(model_path)
list(imported_model.signatures.keys())

['call_item_item']

In [ ]:
result_tensor = imported_model.signatures['call_item_item'](tf.constant([8650774]))
result_tensor

{'output_0': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([ 8941836, 11541077, 11953072,  9109184, 10062595, 12050426,
        11409811, 10357545, 10102320, 11374873, 10867650, 10390716,
         9109088, 11311936, 11691428, 10487343,  9565144,  9071435,
        10555920, 10273738,  8661992, 12746059, 12115410, 10131359,
        11080858, 10371371, 10464977, 11100314, 10375474, 11832342,
         8840743,  9407068, 10581397,  6775544,  9189046, 10234617,
        10437093,  9921383, 10710521,  8768733, 10637116, 11462829,
        11448828, 10104809, 10909750, 12201985,  8927246, 11198985,
        11392806, 12155249, 11493983, 11638705, 10715306, 10357533,
        12111280, 11903387, 10636823, 11054197, 11700318, 11777393,
        10339012,  8469882,  7761008, 12235188, 12259464, 13000935,
        11409896, 12657366, 11211459, 10877926, 11273515, 10260333,
        11427852, 10424246,  9539647, 12420888, 11396817, 11076588,
         9440709, 12738200, 10366394,  8368788, 10327204,

In [ ]:
imported_model.signatures.items()

ItemsView(_SignatureMap({'call_item_item': <ConcreteFunction signature_wrapper(*, product) at 0x7F4616FE7810>}))

In [ ]:
result_tensor = imported_model.signatures['call_item_item'](tf.constant([10308557]))
result_tensor

{'output_0': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([10308557, 10308818, 10309914, 11060961, 10402703, 10565130,
         5801178, 10308540,  7039647, 11344538,  9655345, 10309920,
         9968963,  4275772, 10735521, 10352144, 11251463,  7098445,
         7098451, 10309915, 10352147, 10309782,  9968967,  9775636,
        10563199,  8064082,  9586825, 10637119,  8745086, 10623500,
         7413512, 10854998, 10635879, 10718009, 10366082, 11378268,
         7134751, 10794958,  5373264, 11351422,  9099410,  9875244,
        11801850, 11152568,  8886884, 10366062,  9229342, 10309924,
        12030039,  9225940, 10606090,  5808602,  3921514, 10606080,
         9690295,  8745098, 13112623,  6429322,  9528209,  5580989,
        10988173,  9491423, 12895283, 10564625, 11221174, 11211993,
         7289381, 11433304,  8084941, 12482766, 10353402,  6848491,
         9528191,  7777858, 12286648, 10716895,  9453604, 11141217,
        10636469,  9819136,  9637845, 10581393, 10762456,

In [ ]:
len(products)

29696

In [ ]:
result_tensor = imported_model.signatures['call_item_item'](tf.constant([10329732]))

In [ ]:
result_tensor

{'output_0': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([ 8650774,  9306139,  9961521, 13238328, 10485819, 13238336,
        13238341, 13238349,  9175127, 11141217, 11141222, 11141223,
        11141224, 11141226,  9568368,  8388728,  9568381,  9568388,
         9830537,  9437342,  9961638, 12714153, 13107382, 11403450,
         9044155, 13107389, 10748117, 10748127, 10748128, 10748131,
        10748137, 10748138,  8782062,  8782064, 12058867, 10879220,
        10223863, 10486008, 10879225, 10879227, 10879230, 10879231,
        12321027,  8388869, 10879238, 10879241, 10354955, 10354958,
        10354961, 10354965, 10354966, 10354971, 10354972, 10354973,
         7340321, 10354977, 10354978, 11927847, 13369677,  8520014,
        10223954, 13369684, 10223957, 10223960, 11403612, 11403613,
        11272541, 11796832, 11796833, 11534690, 11272546, 11272553,
        11403626,  8651167, 13107617, 11010469,  8651175, 11010474,
         8651182, 11010484, 12976579, 11010502, 11010507,

In [ ]:
from IPython.core.display import HTML

def path_to_image(path):
  return '<img src = "https://www.asos.com/prd/' + str(path) + '" width = 60 " >'
  


result_df = pd.DataFrame(result_tensor['output_0'].numpy(), columns=['ProductId']).head()
print(result_df.to_html(escape=False, formatters=dict(ProductId = path_to_image)))
HTML(result_df.to_html(escape=False, formatters=dict(ProductId = path_to_image)))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>ProductId</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td><img src = "https://www.asos.com/prd/10308557" width = 60 " ></td>
    </tr>
    <tr>
      <th>1</th>
      <td><img src = "https://www.asos.com/prd/10308818" width = 60 " ></td>
    </tr>
    <tr>
      <th>2</th>
      <td><img src = "https://www.asos.com/prd/10309914" width = 60 " ></td>
    </tr>
    <tr>
      <th>3</th>
      <td><img src = "https://www.asos.com/prd/11060961" width = 60 " ></td>
    </tr>
    <tr>
      <th>4</th>
      <td><img src = "https://www.asos.com/prd/10402703" width = 60 " ></td>
    </tr>
  </tbody>
</table>


,ProductId
0,
1,
2,
3,
4,


In [ ]:
µ#zip the model for simpler use


from zipfile import ZipFile
import os
# create a ZipFile object
with ZipFile('models.zip', 'w') as zipObj:
   # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk("models"):
        for filename in filenames:
           #create complete filepath of file in directory
           filePath = os.path.join(folderName, filename)
           # Add file to zip
           zipObj.write(filePath)



In [ ]:
curl --header "Content-Type: application/json" --request POST --data '{"signature_name":"call_item_item","inputs": {"product": 10093853] } }' http://localhost:8501/v1/models/recommender:predict


SyntaxError: ignored